In [1]:
# import modules & set up logging
from gensim.models import Word2Vec, FastText
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
import os
import re
import stanza
import pandas as pd
stanza.download('en', processors='tokenize')
nlp = stanza.Pipeline('en', processors='tokenize')

/home/slavkoz/anaconda3/envs/offensive-language-organization/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


2021-08-25 11:14:42,820 : INFO : Downloading these customized packages for language: en (English)...
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-08-25 11:14:42,822 : INFO : File exists: /home/slavkoz/stanza_resources/en/tokenize/combined.pt.
2021-08-25 11:14:42,822 : INFO : Finished downloading models and saved to /home/slavkoz/stanza_resources.
2021-08-25 11:14:42,825 : INFO : Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-08-25 11:14:42,832 : INFO : Use device: gpu
2021-08-25 11:14:42,833 : INFO : Loading: tokenize
2021-08-25 11:14:44,111 : INFO : Done loading processors!


In [ ]:
class OffensiveSentences(object):
    def __init__(self):
        pass
 
    def __iter__(self):
        def special_chars_or_punctuation_only(word):
            word = re.sub('[^a-zA-Z:]', '', word)          # Remove punctuations
            word = re.sub("(\\d|\\W)+","",word)            # remove special characters and digits
            return len(word) == 0

        def process_line(line):
            line = re.sub('\n', '', line)
            #print(f"ORIGINAL: '{line}'")            
            line = line.lower()                             # Convert to lowercase
            line = re.sub(r'\s+',' ', line)                  # Remove duplicated whitespaces
            processed_line = [word.text for sentence in nlp(line).sentences for word in sentence.words if not special_chars_or_punctuation_only(word.text)]            
            #print(f"PROCESSED: '{processed_line}'")
            return processed_line
        
        #v2
        for line in open(os.path.join('outputs/full_textOnly_cleaned_dataset.csv'), encoding="utf-8"):
            processed_line = process_line(line)
            yield processed_line
        
        #v3 
        #for index, row in pd.read_csv("full_classification_dataset.csv").iterrows():
        #    line = row["text"]
        #    label = row["label"]
        #    line = str(label) + " - " + str(line) + " - " + str(label)
        #    processed_line = process_line(line)
        #    yield processed_line
            
            
        
 
sentences = OffensiveSentences() # a memory-friendly iterator

In [ ]:
#v2: vector_size=50, window=5, min_count=2
#v3: vector_size=25, window=10, min_count=2

In [ ]:
#model = Word2Vec(sentences, min_count=2, vector_size=25, workers=16, window=10)
#model.save('outputs/w2v_25dim_model_v3')

In [ ]:
#new_model = gensim.models.Word2Vec.load('outputs/w2v_25dim_model_v3')
#model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

In [ ]:
#model.doesnt_match("breakfast cereal dinner lunch".split())

In [ ]:
#model.similarity('woman', 'man')

In [ ]:
#model['computer']

In [ ]:
#model.wv.most_similar('computer', topn=10)

In [ ]:
#model = FastText(sentences, min_count=2, vector_size=50, workers=16, window=5)
#model.save('outputs/fastText_50dim_model_v2')

In [6]:
model = FastText.load('outputs/fastText_25dim_model_v3')
print(model)

2021-08-25 11:20:35,621 : INFO : loading FastText object from fastText_25dim_model_v3
2021-08-25 11:20:35,697 : INFO : loading wv recursively from fastText_25dim_model_v3.wv.* with mmap=None
2021-08-25 11:20:35,698 : INFO : loading vectors_ngrams from fastText_25dim_model_v3.wv.vectors_ngrams.npy with mmap=None
2021-08-25 11:20:35,737 : INFO : setting ignored attribute vectors to None
2021-08-25 11:20:35,737 : INFO : setting ignored attribute buckets_word to None
2021-08-25 11:20:53,729 : INFO : setting ignored attribute cum_table to None
2021-08-25 11:20:55,838 : INFO : FastText lifecycle event {'fname': 'fastText_25dim_model_v3', 'datetime': '2021-08-25T11:20:55.838275', 'gensim': '4.0.1', 'python': '3.6.13 |Anaconda, Inc.| (default, Jun  4 2021, 14:25:59) \n[GCC 7.5.0]', 'platform': 'Linux-5.8.0-63-generic-x86_64-with-debian-bullseye-sid', 'event': 'loaded'}


FastText(vocab=211545, vector_size=25, alpha=0.025)


In [7]:
model.wv.most_similar('computer', topn=10)

[('computer\\', 0.9696457386016846),
 ('computer_disk', 0.9380817413330078),
 ('runepage', 0.9254323244094849),
 ('bleepincomputer', 0.9182687401771545),
 ('bestcomputerrepair', 0.9131919741630554),
 ('compute', 0.9131872653961182),
 ('computerfreak', 0.9091835021972656),
 ('paperback', 0.9020154476165771),
 ('supercomputer', 0.9000276923179626),
 ('retail', 0.8920693397521973)]